In [11]:
from pyspark.sql import (
    functions as f,
    SparkSession,
    types as t
)

spark = SparkSession.builder.appName("df_missing_data").getOrCreate()
df = spark.read.csv(
    "file:///home/jovyan/work/sample/null_data.csv", header=True, inferSchema=True)
df.show()

+----+----------+------+
|  id|occupation|salary|
+----+----------+------+
|1000|  engineer|100000|
|2000|      NULL|  NULL|
|3000|      NULL| 50000|
|4000|   teacher| 80000|
|5000|    banker|120000|
+----+----------+------+



In [12]:
# DataFrame.na: Returns a DataFrameNaFunctions for handling missing values.
# DataFrame.dropna(how='any', thresh=None, subset=None)[source]: Returns a new DataFrame omitting rows with null values. DataFrame.dropna() and DataFrameNaFunctions.drop() are aliases of each other.
#   how: 'any’ or ‘all’. If ‘any’, drop a row if it contains any nulls. If ‘all’, drop a row only if all its values are null.
#   thresh: default None If specified, drop rows that have less than thresh non-null values. This overwrites the how parameter.
#   subset: optional list of column names to consider.

# df.na.drop(how="any").show()
# df.na.drop(thresh=2).show()
# df.na.drop(subset=["salary"]).show()

df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- occupation: string (nullable = true)
 |-- salary: integer (nullable = true)



In [13]:
# # fill string
# df.na.fill("engineer").show()

# # fill integer
# df.na.fill(0).show()

# # fill the subset
df.na.fill("NA", subset=["occupation"]).show()

# # fill the mean value
mean_value = df.select(f.mean(df['salary'])).collect()

# print(mean_value[0][0])

df.na.fill(mean_value[0][0], subset=["salary"]).show()

+----+----------+------+
|  id|occupation|salary|
+----+----------+------+
|1000|  engineer|100000|
|2000|        NA|  NULL|
|3000|        NA| 50000|
|4000|   teacher| 80000|
|5000|    banker|120000|
+----+----------+------+

+----+----------+------+
|  id|occupation|salary|
+----+----------+------+
|1000|  engineer|100000|
|2000|      NULL| 87500|
|3000|      NULL| 50000|
|4000|   teacher| 80000|
|5000|    banker|120000|
+----+----------+------+



In [14]:
# Date parsing
spark = SparkSession.builder.appName("df_manage_date").getOrCreate()
df = spark.read.csv(
    "file:///home/jovyan/work/sample/date_parsing.csv", header=True, inferSchema=True)

# # show year
df.select(f.year('date')).show()

# # show month
df.select(f.month('date')).show()

+----------+
|year(date)|
+----------+
|      2022|
|      2021|
|      2022|
|      2022|
|      2022|
|      2022|
|      2021|
|      2022|
|      2021|
|      2021|
|      2022|
|      2022|
|      2022|
|      2022|
|      2022|
|      2022|
|      2022|
|      2022|
|      2022|
|      2022|
+----------+
only showing top 20 rows

+-----------+
|month(date)|
+-----------+
|          1|
|         12|
|          1|
|          1|
|          4|
|         10|
|         12|
|          5|
|         12|
|         12|
|          5|
|          6|
|          4|
|          8|
|          1|
|          9|
|          5|
|          9|
|          7|
|          7|
+-----------+
only showing top 20 rows



In [15]:
# # show day
df.select(f.dayofmonth('date').alias('day')).show()
df.select(f.dayofyear('date').alias('day')).show()

df = df.withColumn("year", f.year('date')).groupBy("year").mean("number").withColumnRenamed("avg(number)", "avg")
# df.show()
df.select("year", f.format_number("avg", 2).alias("avg")).show()

+---+
|day|
+---+
| 27|
| 29|
| 22|
|  6|
| 21|
| 23|
| 23|
| 31|
| 29|
| 30|
|  4|
| 22|
| 23|
|  4|
| 26|
| 23|
| 27|
| 20|
|  5|
| 18|
+---+
only showing top 20 rows

+---+
|day|
+---+
| 27|
|363|
| 22|
|  6|
|111|
|296|
|357|
|151|
|363|
|364|
|124|
|173|
|113|
|216|
| 26|
|266|
|147|
|263|
|186|
|199|
+---+
only showing top 20 rows

+----+--------+
|year|     avg|
+----+--------+
|2022|2,540.67|
|2021|2,195.68|
+----+--------+

